<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/notion_expense_sync_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ 1. ติดตั้งไลบรารีที่ต้องใช้
!pip install gspread oauth2client notion-client python-dotenv

In [ ]:
# ✅ 2. Mount Google Drive และโหลด .env
from google.colab import drive
drive.mount('/content/drive')

from dotenv import load_dotenv
import os

env_path = '/content/drive/MyDrive/Automation_Keys/Notion_Expense/.env'
load_dotenv(dotenv_path=env_path)

# ✅ โหลดค่าจาก .env
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_DB_ID = os.getenv("NOTION_DB_ID")
SHEET_ID = os.getenv("SHEET_ID")

In [ ]:
# ✅ 3. เชื่อม Google Sheets ด้วย service_account.json ที่อยู่ใน Google Drive
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# ✅ เปลี่ยนชื่อไฟล์ใน Drive เป็น service_account.json แล้ว
service_account_path = "/content/drive/MyDrive/Automation_Keys/Notion_Expense/service_account.json"

creds = ServiceAccountCredentials.from_json_keyfile_name(service_account_path, scope)
client = gspread.authorize(creds)

# ✅ เปิด Sheet แรกของไฟล์ด้วย SHEET_ID จาก .env
sheet = client.open_by_key(SHEET_ID).sheet1

In [ ]:
# ✅ 4. ฟังก์ชันแปลงข้อความ [หมวดหมู่] [จำนวนเงิน]
def parse_message(text):
    try:
        parts = text.strip().split()
        if len(parts) == 2:
            category = parts[0]
            amount = float(parts[1])
            return category, amount
        elif len(parts) == 1:
            return parts[0], 0.0  # เผื่อพิมพ์หมวดหมู่อย่างเดียว
        else:
            return None, None
    except:
        return None, None

In [ ]:
# ✅ 5. เชื่อมต่อ Notion API
from notion_client import Client

notion = Client(auth=NOTION_TOKEN)

def push_to_notion(category, amount):
    notion.pages.create(
        parent={"database_id": NOTION_DB_ID},
        properties={
            "Categories": {"select": {"name": category}},
            "Amount": {"number": amount}
        }
    )

In [ ]:
# ✅ 6. ฟังก์ชัน Sync Google Sheet → Notion
def sync_sheet_to_notion():
    data = sheet.get_all_records()
    for idx, row in enumerate(data):
        if row['Synced?'].strip().upper() != "NO":
            continue

        category, amount = parse_message(row['Message'])
        if not category or amount is None:
            print(f"❌ Format ผิดที่ row {idx+2}: {row['Message']}")
            continue

        try:
            push_to_notion(category, amount)
            sheet.update_cell(idx+2, 4, "YES")
            print(f"✅ Synced row {idx+2}: {category} {amount}")
        except Exception as e:
            print(f"🚨 Error at row {idx+2}: {e}")

In [ ]:
# ✅ 7. บันทึกข้อความจาก Line ลง Google Sheet
import datetime

def log_message(user_id, message):
    now = datetime.datetime.now().isoformat()
    sheet.append_row([now, user_id, message, "NO"])  # เพิ่มแถวใหม่ลง Google Sheet